# Project 1 Portfolio Analyzer Consumer Stocks VS. Tech Stocks

We will be projecting future returns of a portfolio comprised of five consumer stocks as well as a a portfolio comprised of 5 tech stocks over a 15 and 20 year period in order to evaluate the better long term buy. The asset allocation between between stocks and bonds will remain the same between both portfolios.


The Stocks we will be analyzing are 

# Consumer Stocks:             

* P&G
* Johnson and Johnson
* Walmart
* Kellogg
* Coca Cola

# Tech Stocks:

* Amazon 
* Meta
* Google
* Tesla
* NVIDIA



In [1]:
import pandas as pd
import yfinance as yf
from pathlib import Path
import os
import requests
import json
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from MCForecastTools import MCSimulation
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
##pip install PyPortfolioOpt
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from matplotlib.ticker import FuncFormatter

%matplotlib inline

ModuleNotFoundError: No module named 'dotenv'

## Cost Basis: $50,000

In [ ]:
all_stocks = pd.read_csv('Resources/newstocks.csv', header=1)
all_stocks= all_stocks.iloc[:, 0:12]
all_stocks = all_stocks.rename(columns={"Unnamed: 0":"Date"})
all_stocks = all_stocks.drop(all_stocks.index[0])
all_stocks = all_stocks.set_index("Date")
all_stocks = all_stocks.fillna(0)
all_stocks.head()

In [ ]:
tech_stocks = all_stocks.loc[:, ['AAPL','AMZN', 'GOOGL','MSFT','NVDA']]
AVG_TECH = pd.DataFrame()
AVG_TECH['AVG_TECH'] = (tech_stocks.iloc[:, 0] + tech_stocks.iloc[:, 1] + tech_stocks.iloc[:, 2] + tech_stocks.iloc[:, 3] + tech_stocks.iloc[:, 4])/5
display(AVG_TECH.head())
display(tech_stocks.head())


In [ ]:
consumer_stocks = all_stocks.loc[:, ['JNJ','K','KO','PG','WMT']]
AVG_CONS = pd.DataFrame()
AVG_CONS['AVG_CONS'] = (consumer_stocks.iloc[:, 0] + consumer_stocks.iloc[:, 1] + consumer_stocks.iloc[:, 2] + consumer_stocks.iloc[:, 3] + consumer_stocks.iloc[:, 4])/5
display(AVG_CONS.head())
display(consumer_stocks.head())


In [ ]:
sp_500 = all_stocks.loc[:, ['^GSPC']]
sp_500.head()

In [ ]:
tech_stocks.plot(figsize=(20,6), title='Share Price from 2005 to 2023', legend=True, ylabel= "Share Price")

In [ ]:
consumer_stocks.plot(figsize=(20,6), title='Share Price from 2005 to 2023', legend=True, ylabel= "Share Price")

In [ ]:
ax = AVG_TECH.plot(figsize=(20,6), title='Average Share Price from 2005 to 2023', legend=True, ylabel= "Share Price")
AVG_CONS.plot(ax=ax)

# Daily Returns Tech Stocks

In [ ]:
display(tech_stocks.head())
daily_returns_tech_stocks = tech_stocks.pct_change()
daily_returns_tech_stocks = daily_returns_tech_stocks.fillna(0)
display(daily_returns_tech_stocks.head())
daily_returns_18_year_tech = daily_returns_tech_stocks.loc["2005-01-03":"2023-04-03"]
daily_returns_18_year_tech.describe()

# Daily Returns Average for Tech Stocks

In [ ]:
display(AVG_TECH.head())
daily_returns_avgtech_stocks = AVG_TECH.pct_change()
daily_returns_avgtech_stocks = daily_returns_avgtech_stocks.fillna(0)
display(daily_returns_avgtech_stocks.head())
daily_returns_18_year_avgtech = daily_returns_avgtech_stocks.loc["2005-01-03":"2023-04-03"]
display(daily_returns_18_year_avgtech.describe())

# Daily Returns Consumer Stocks

In [ ]:
display(consumer_stocks.head())
daily_returns_consumer_stocks = consumer_stocks.pct_change()
daily_returns_consumer_stocks = daily_returns_consumer_stocks.fillna(0)
display(daily_returns_consumer_stocks.head())
daily_returns_18_year_consumer = daily_returns_consumer_stocks.loc["2005-01-03":"2023-04-03"]
daily_returns_18_year_consumer.describe()

# Daily Returns Average for Consumer Stocks

In [ ]:
display(AVG_CONS.head())
daily_returns_avgconsumer_stocks = AVG_CONS.pct_change()
daily_returns_avgconsumer_stocks = daily_returns_avgconsumer_stocks.fillna(0)
display(daily_returns_avgconsumer_stocks.head())
daily_returns_18_year_avgconsumer = daily_returns_avgconsumer_stocks.loc["2005-01-03":"2023-04-03"]
daily_returns_18_year_avgconsumer.describe()

# Daily Returns SP500

In [ ]:
display(sp_500.head())
daily_returns_sp_500 = sp_500.pct_change()
daily_returns_sp_500 = daily_returns_sp_500.fillna(0)
display(daily_returns_sp_500.head())
daily_returns_18_year_sp_500 = daily_returns_sp_500.loc["2005-01-03":"2023-04-03"]
daily_returns_18_year_sp_500.describe()

In [ ]:
cumulative_returns_tech = (1 + daily_returns_tech_stocks).cumprod()
cumulative_returns_tech.plot(figsize=(20,6), title="Large Cap Tech Stocks Cumulative Returns from 2005-2023")

In [ ]:
cumulative_returns_consumer = (1 + daily_returns_consumer_stocks).cumprod()
cumulative_returns_consumer.plot(figsize=(20,6), title="Large Cap Consumer Stocks Cumulative Returns from 2005-2023")

In [ ]:
cumulative_returns_avgtech = (1 + daily_returns_avgtech_stocks).cumprod()
cumulative_returns_avgconsumer = (1 + daily_returns_avgconsumer_stocks).cumprod()
cumulative_returns_sp_500 = (1 + daily_returns_sp_500).cumprod()
ax = cumulative_returns_avgtech.plot(figsize=(20,6), title="Large Cap Stocks Cumulative Returns from 2005-2023")
cumulative_returns_avgconsumer.plot(ax=ax)
cumulative_returns_sp_500.plot(ax=ax)

# Variance For Tech Stocks

In [ ]:
aapl_variance = daily_returns_tech_stocks['AAPL'].var()
amzn_variance = daily_returns_tech_stocks['AMZN'].var()
googl_variance = daily_returns_tech_stocks['GOOGL'].var()
msft_variance = daily_returns_tech_stocks['MSFT'].var()
nvda_variance = daily_returns_tech_stocks['NVDA'].var()
display(aapl_variance, amzn_variance, googl_variance, msft_variance, nvda_variance)

# Covariance for Tech Stocks

In [ ]:
aapl_cov = daily_returns_tech_stocks['AAPL'].cov(daily_returns_sp_500['^GSPC'])
amzn_cov = daily_returns_tech_stocks['AMZN'].cov(daily_returns_sp_500['^GSPC'])
googl_cov = daily_returns_tech_stocks['GOOGL'].cov(daily_returns_sp_500['^GSPC'])
msft_cov = daily_returns_tech_stocks['MSFT'].cov(daily_returns_sp_500['^GSPC'])
nvda_cov = daily_returns_tech_stocks['NVDA'].cov(daily_returns_sp_500['^GSPC'])
display(aapl_cov, amzn_cov, googl_cov, msft_cov, nvda_cov)


# Beta For Tech Stocks

In [ ]:
aapl_beta = aapl_cov / aapl_variance
amzn_beta = amzn_cov / amzn_variance
googl_beta = googl_cov / googl_variance
msft_beta = msft_cov / msft_variance
nvda_beta = nvda_cov / nvda_variance
display(aapl_beta, amzn_beta, googl_beta, msft_beta, nvda_beta)

# Rolling Metrics for Tech Stocks

In [ ]:
daily_returns_tech_stocks.rolling(window=180).mean().plot(figsize=(15,5))

# Variance For Consumer Stocks

In [ ]:
jnj_variance = daily_returns_consumer_stocks['JNJ'].var()
jnj_variance

# Covariance For Consumer Stocks

In [ ]:
jnj_cov = daily_returns_consumer_stocks['JNJ'].cov(daily_returns_sp_500['^GSPC'])
jnj_cov

# Beta For Consumer Stocks

In [ ]:
jnj_beta = jnj_cov / jnj_variance
jnj_beta

# Rolling Metrics For Consumer Stocks

In [ ]:
daily_returns_consumer_stocks.rolling(window=180).mean().plot(figsize=(15,5))

# SP500 and Tech

In [ ]:
ax = daily_returns_tech_stocks.plot()
daily_returns_sp_500.plot(ax=ax)

# SP500 and Consumer

In [ ]:
ax = daily_returns_consumer_stocks.plot()
daily_returns_sp_500.plot(ax=ax)

## Tech Stock Portfolio Simulation

In [ ]:
## Portfolio Asset Allocation:

load_dotenv()

alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")
type(alpaca_secret_key )


alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version = "v2"
)

tickers = ["AGG","AMZN", "GOOGL", "AAPL", "MSFT", "NVDA"]

timeframe = "1DAY"

start_date = pd.Timestamp("2013-04-05", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2023-04-05", tz="America/New_York").isoformat()

tech_portfolio = alpaca.get_bars(
    tickers,
    timeframe,
    start = start_date,
    end = end_date
).df

AGG = tech_portfolio[tech_portfolio['symbol']=='AGG'].drop('symbol', axis=1)

AMZN = tech_portfolio[tech_portfolio['symbol']=='AMZN'].drop('symbol', axis=1)

GOOGL = tech_portfolio[tech_portfolio['symbol']=='GOOGL'].drop('symbol', axis=1)

AAPL = tech_portfolio[tech_portfolio['symbol']=='AAPL'].drop('symbol', axis=1)

MSFT = tech_portfolio[tech_portfolio['symbol']=='MSFT'].drop('symbol', axis=1)

NVDA = tech_portfolio[tech_portfolio['symbol']=='NVDA'].drop('symbol', axis=1)

tech_portfolio = pd.concat([AGG,AMZN,GOOGL,AAPL,MSFT,NVDA],axis=1, keys=['AGG','AMZN','GOOGL','AAPL','MSFT','NVDA'])

tech_portfolio.head()

In [ ]:
agg_close_price = tech_portfolio["AGG"]["close"]

print(agg_close_price[-1])

AAPL_close_price = tech_portfolio["AAPL"]["close"]

print(AAPL_close_price[-1])

AMZN_close_price = tech_portfolio["AMZN"]["close"]

print(AMZN_close_price[-1])

GOOGL_close_price = tech_portfolio["GOOGL"]["close"]

print(GOOGL_close_price[-1])

MSFT_close_price = tech_portfolio["MSFT"]["close"]

print(MSFT_close_price[-1])

NVDA_close_price = tech_portfolio["NVDA"]["close"]

print(NVDA_close_price[-1])


In [ ]:
MC_15year_tech = MCSimulation(
    portfolio_data = tech_portfolio,
    weights = [.20,.16,.16,.16,.16,.16],
    num_simulation = 500,
    num_trading_days = 252*15)

MC_15year_tech.calc_cumulative_return()

In [ ]:
MC_15year_tech.plot_distribution()

In [ ]:
MC_15year_tech_summary_statistics = MC_15year_tech.summarize_cumulative_return()

display(MC_15year_tech_summary_statistics)

total_tech_stocks_bonds = 50000

ci_lower_15_cumulative_return_tech = MC_15year_tech_summary_statistics[8]*total_tech_stocks_bonds
ci_upper_15_cumulative_return_tech = MC_15year_tech_summary_statistics[9]*total_tech_stocks_bonds

print(f"There is a 95% chance that an initial investment of ${total_tech_stocks_bonds} in the portfolio over the next 15 years will end within the range of"
        f" ${ci_lower_15_cumulative_return_tech} and ${ci_upper_15_cumulative_return_tech}.")

## Consumer stocks 

In [ ]:
load_dotenv()

alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")
type(alpaca_secret_key )


alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version = "v2"
)

tickers = ["AGG","JNJ", "K", "KO", "PG", "WMT"]

timeframe = "1DAY"

start_date = pd.Timestamp("2013-04-05", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2023-04-05", tz="America/New_York").isoformat()

consumer_portfolio = alpaca.get_bars(
    tickers,
    timeframe,
    start = start_date,
    end = end_date
).df

AGG = consumer_portfolio[consumer_portfolio['symbol']=='AGG'].drop('symbol', axis=1)

JNJ = consumer_portfolio[consumer_portfolio['symbol']=='JNJ'].drop('symbol', axis=1)

K = consumer_portfolio[consumer_portfolio['symbol']=='K'].drop('symbol', axis=1)

KO = consumer_portfolio[consumer_portfolio['symbol']=='KO'].drop('symbol', axis=1)

PG = consumer_portfolio[consumer_portfolio['symbol']=='PG'].drop('symbol', axis=1)

WMT = consumer_portfolio[consumer_portfolio['symbol']=='WMT'].drop('symbol', axis=1)

consumer_portfolio = pd.concat([AGG,JNJ,K,KO,PG,WMT],axis=1, keys=['AGG','JNJ','K','KO','PG','WMT'])

consumer_portfolio.head()

In [ ]:
agg_close_price = consumer_portfolio["AGG"]["close"]

print(f"The closing price for AGG was ${agg_close_price[-1]}.")

JNJ_close_price = consumer_portfolio["JNJ"]["close"]

print(f"The closing price for Johnson and Johnson was ${JNJ_close_price[-1]}.")

K_close_price = consumer_portfolio["K"]["close"]

print(f"The closing price for Kellogg was ${K_close_price[-1]}.")

KO_close_price = consumer_portfolio["KO"]["close"]

print(f"The closing price for Coca Cola was ${KO_close_price[-1]}.")

PG_close_price = consumer_portfolio["PG"]["close"]

print(f"The closing price for Proctor and Gamble was ${PG_close_price[-1]}.")

WMT_close_price = consumer_portfolio["WMT"]["close"]

print(f"The closing price for Walmart was ${WMT_close_price[-1]}.")

print("Given the above closing prices and the equal asset allocation amongst all the consumer stocks using a cost basis of $50,000, the following are the number of shares")

In [ ]:
MC_15year_consumer = MCSimulation(
    portfolio_data = tech_portfolio,
    weights = [.20,.16,.16,.16,.16,.16],
    num_simulation = 500,
    num_trading_days = 252*15)

MC_15year_consumer.calc_cumulative_return()

MC_15year_consumer.plot_distribution()

In [ ]:
MC_15year_consumer_summary_statistics = MC_15year_consumer.summarize_cumulative_return()

display(MC_15year_consumer_summary_statistics)

In [ ]:
total_consumer_stocks_bonds = 50000
ci_lower_15_cumulative_return_consumer = MC_15year_consumer_summary_statistics[8]*total_consumer_stocks_bonds
ci_upper_15_cumulative_return_consumer = MC_15year_consumer_summary_statistics[9]*total_consumer_stocks_bonds

print(f"There is a 95% chance that an initial investment of {total_consumer_stocks_bonds} in the portfolio over the next 15 years will end within the range of"
        f" ${ci_lower_15_cumulative_return_consumer} and ${ci_upper_15_cumulative_return_consumer}.")

## Analysis of Portfolio Valuations:

#Need annualized returns, std dev for annualized returns portfolio a and b

In [ ]:
MC_15year_consumer_daily = MC_15year_consumer.calculate_daily_return()
display(MC_15year_consumer_daily)

## Portfolio C:
## How to Select Stocks in Portfolio - USE CAPM model - MSFT, KO, AAPL, PG, AMZN?

In [ ]:
tickers = ['MSFT','KO','AAPL','PG','AMZN']
len_tickers = len(tickers)
price_data = []
for ticker in range(len_tickers):
   prices = web.DataReader(tickers[ticker], start='2018-06-20', end = '2020-06-20', data_source='yahoo')
   price_data.append(prices[['Adj Close']])
df_stocks = pd.concat(price_data, axis=1)
df_stocks.columns=tickers
df_stocks.tail()

## TASKS:

## Calculate Performance Metrics

## Analysis

## Portfolio C: Which 5 stocks out of these 10 would give the highest risk-adjusted return? (Highest Ratios)

## CAPM